In [1]:
%serialconnect --port=/dev/ttyS15

Connecting to --port=/dev/ttyS15 --baud=115200 
Ready.


In [2]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.12-68-g3032ae115 on 2020-01-19; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nMPY: soft reboot\r\nMicroPython v1.12-68-g3032ae115 on 2020-01-19; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [3]:
import machine
import time
import struct

In [4]:
sck = machine.Pin(18,machine.Pin.OUT)
mosi = machine.Pin(23,machine.Pin.OUT)
miso = machine.Pin(19,machine.Pin.IN)
cs = machine.Pin(5,machine.Pin.OUT)

In [5]:
spi = machine.SPI(1,baudrate=200000,bits=8,polarity=1,phase=1,firstbit=machine.SPI.MSB,sck=sck,mosi=mosi,miso=miso)

## SPI Write Command
```cpp
void SPI_writeCommand(byte command) {
  digitalWrite(slaveSelectPin, LOW); //SS goes low to mark start of transmission
  union FourByte data = {0xFEFEFE, command}; //generate the data to be sent, i.e. your command plus the Sync bytes.
  // Serial1.print("SPI_writeCommand");
  for (int i = 3; i >= 0; i--) {
    SPI.transfer(data.bit8[i]); //transfer all 4 bytes of data - command first, then Big Endian transfer of the 24bit value.
    // Serial1.print(i);
    // Serial1.print(data.bit8[i], HEX);
  }
  // Serial1.println();
  digitalWrite(slaveSelectPin, HIGH);
}
```

## SPI Read Command
```cpp
// Read a register from the CS5464 - just supply a command byte (see the datasheet)
unsigned long SPI_read(byte command) {
  digitalWrite(slaveSelectPin, LOW); //SS goes low to mark start of transmission
  union FourByte data = {0xFEFEFE, command}; //generate the data to be sent, i.e. your command plus the Sync bytes.
  // Serial1.print("SPI_Read");
  for (int i = 3; i >= 0; i--) {
    data.bit8[i] = SPI.transfer(data.bit8[i]); //send the data whilst reading in the result
    // Serial1.print(data.bit8[i], HEX);
  }
  // Serial1.println();
  digitalWrite(slaveSelectPin, HIGH); //SS goes high to mark end of transmission
  return data.value; //return the 24bit value recieved.
}
```

In [6]:
def write_command(command,cs,spi):
    cs.value(False)
    otw_value = struct.pack('>BBBB',command,0xFE,0xFE,0xFE)
    spi.write(otw_value)
    result = spi.read(4)
    cs.value(True)
    return result

def read_command(command,cs,spi):
    cs.value(False)
    otw_value = struct.pack('>BBBB',command,0xFE,0xFE,0xFE)
    spi.write(otw_value)
    result = spi.read(4)
    cs.value(True)
    return result

In [7]:
reset = write_command(0x80,cs,spi)
print(reset)

b'\x00\x00\x00\x00'


In [8]:
status = read_command(0b00011110,cs,spi)
print(status)
status = struct.unpack('>L',status)[0]

b'\x00\x00\x00\x00'


In [9]:
print(status)

0


In [10]:
print((1<<23) & (status))

0


In [11]:
mode = write_command(0xE8, cs, spi)

In [12]:
print(mode)

b'\x00\x00\x00\x00'


In [13]:
current = read_command(0b00101100,cs,spi)

In [14]:
print(current)

b'\x00\x00\x00\x00'
